## Obtendo as Demonstrações Financeiras Padronizadas

In [ ]:
install.packages("purrr")
install.packages("httr")
install.packages("lubridate")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(httr)
library(purrr)
library(dplyr)
library(tidyverse)
library(lubridate)
#library(RPostgreSQL)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ readr     2.1.5     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [ ]:
obter_dfp <- function(ano){
  url <- paste0("https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/","dfp_cia_aberta_",ano,".zip") #copiando a URL dos arquivos
  GET(url, write_disk(paste0("dfp_cia_aberta_",ano,".zip"))) #baixando os arquivos
  planilhas <- unzip(paste0("/content/dfp_cia_aberta_",ano,".zip"))} #descompactando os arquivos .RAR

In [ ]:
cvm_dfp <- function(){
  #Listando os arquivos no diretório referente às demonstrações financeiras
  BPA <- list.files(path = ".", pattern = "^dfp_cia_aberta_BPA_con_", full.names = TRUE)
  BPP <- list.files(path = ".", pattern = "^dfp_cia_aberta_BPP_con_", full.names = TRUE)
  DRE <- list.files(path = ".", pattern = "^dfp_cia_aberta_DRE_con_", full.names = TRUE)
  DFC <- list.files(path = ".", pattern = "^dfp_cia_aberta_DFC_MD_con_", full.names = TRUE)

  #Lendo os arquivos todos de uma vez para cada demonstração
  #Quando necessário: advinhar o encoding usar a função readr::guess_encoding
  BPA <- read_csv2(BPA, id = "file_name",locale = locale(encoding = "ISO-8859-1"))
  BPP <- read_csv2(BPP, id = "file_name",locale = locale(encoding = "ISO-8859-1"))
  DRE <- read_csv2(DRE, id = "file_name",locale = locale(encoding = "ISO-8859-1"))
  DFC <- read_csv2(DFC, id = "file_name",locale = locale(encoding = "ISO-8859-1"))

  #Balanço Patrimonial - Ativo
  BPA <- BPA %>% filter(ORDEM_EXERC == "ÚLTIMO" & ST_CONTA_FIXA == "S") %>%
    select(DT_REFER,DENOM_CIA,CD_CVM,GRUPO_DFP,ESCALA_MOEDA,ORDEM_EXERC,DS_CONTA,VL_CONTA) %>%
    mutate(VL_CONTA = as.double(VL_CONTA), GRUPO_DFP = str_replace_all(GRUPO_DFP,"DF Consolidado - Balanço Patrimonial Ativo","BPA")) %>%
    filter(DS_CONTA %in% c("Ativo Total", "Caixa e Equivalentes de Caixa","Aplicações Financeiras","Empréstimos e Recebíveis",
                           "Tributos Diferidos","Outros Ativos","Investimentos","Imobilizado","Intangível","Ativo Circulante","Ativo Não Circulante") &
             VL_CONTA>=0) %>%
    mutate(DS_CONTA = str_replace_all(DS_CONTA,c("Caixa e Equivalentes de Caixa"="Ativo Circulante","Aplicações Financeiras"="Ativo Não Circulante",
                                                 "Empréstimos e Recebíveis"="Ativo Não Circulante","Tributos Diferidos"="Ativo Não Circulante",
                                                 "Outros Ativos"="Ativo Não Circulante","Investimentos"="Ativo Não Circulante","Imobilizado"="Ativo Não Circulante",
                                                 "Intangível"="Ativo Não Circulante"))) %>%
    select(DT_REFER, DENOM_CIA, CD_CVM, GRUPO_DFP, DS_CONTA, VL_CONTA)

  #Balanço Patrimonial - Passivo
  BPP <- BPP %>% filter(ORDEM_EXERC == "ÚLTIMO" & ST_CONTA_FIXA == "S") %>%
    select(DT_REFER,DENOM_CIA,CD_CVM,GRUPO_DFP,ESCALA_MOEDA,ORDEM_EXERC,DS_CONTA,VL_CONTA) %>%
    mutate(VL_CONTA = as.double(VL_CONTA), GRUPO_DFP = str_replace_all(GRUPO_DFP,"DF Consolidado - Balanço Patrimonial Passivo","BPP")) %>%
    filter(DS_CONTA %in% c("Passivo Total","Passivos Financeiros para Negociação","Outros Passivos Financeiros ao Valor Justo no Resultado",
                           "Passivos Financeiros ao Custo Amortizado","Provisões","Passivos Fiscais","Outros Passivos","Passivos sobre Ativos Não Correntes a Venda e Descontinuados",
                           "Patrimônio Líquido Consolidado","Passivo Circulante","Passivo Não Circulante") &
             VL_CONTA>=0) %>%
    mutate(DS_CONTA = str_replace_all(DS_CONTA, c("Passivos Financeiros para Negociação"="Passivo Circulante","Outros Passivos Financeiros ao Valor Justo no Resultado"="Passivo Circulante",
                                                  "Passivos Financeiros ao Custo Amortizado"="Passivo Circulante","Provisões"="Passivo Não Circulante","Passivos Fiscais"="Passivo Não Circulante",
                                                  "Outros Passivos"="Passivo Não Circulante","Passivos sobre Ativos Não Correntes a Venda e Descontinuados"="Passivo Não Circulante",
                                                  "Patrimônio Líquido Consolidado"="Patrimônio Líquido"))) %>%
    select(DT_REFER, DENOM_CIA, CD_CVM, GRUPO_DFP, DS_CONTA, VL_CONTA)

  #Demonstração de Resultado
  DRE <- DRE %>% filter(ORDEM_EXERC == "ÚLTIMO" & ST_CONTA_FIXA == "S") %>%
    select(DT_REFER,DENOM_CIA,CD_CVM,GRUPO_DFP,ESCALA_MOEDA,ORDEM_EXERC,DS_CONTA,VL_CONTA) %>%
    mutate(VL_CONTA = as.double(VL_CONTA), GRUPO_DFP = str_replace_all(GRUPO_DFP,"DF Consolidado - Demonstração do Resultado","DRE")) %>%
    filter(DS_CONTA %in% c("Receitas da Intermediação Financeira","Resultado Antes dos Tributos sobre o Lucro","Lucro/Prejuízo Consolidado do Período",
                           "Receita de Venda de Bens e/ou Serviços","Resultado Antes do Resultado Financeiro e dos Tributos",
                           "Receitas das Operações")) %>%
    mutate(DS_CONTA = str_replace_all(DS_CONTA, c("Receitas da Intermediação Financeira"="Receitas","Receita de Venda de Bens e/ou Serviços"="Receitas","Receitas das Operações"="Receitas",
                                                  "Resultado Antes dos Tributos sobre o Lucro"="Resultado Operacional","Resultado Antes do Resultado Financeiro e dos Tributos"="Resultado Operacional",
                                                  "Lucro/Prejuízo Consolidado do Período"="Lucro Líquido"))) %>%
    select(DT_REFER, DENOM_CIA, CD_CVM, GRUPO_DFP, DS_CONTA, VL_CONTA)

  #Demonstração do Fluxo de Caixa
  DFC <- DFC %>% filter(ORDEM_EXERC == "ÚLTIMO" & ST_CONTA_FIXA == "S") %>%
    select(DT_REFER,DENOM_CIA,CD_CVM,GRUPO_DFP,ESCALA_MOEDA,ORDEM_EXERC,DS_CONTA,VL_CONTA) %>%
    mutate(VL_CONTA = as.double(VL_CONTA), GRUPO_DFP = str_replace_all(GRUPO_DFP,"DF Consolidado - Demonstração do Fluxo de Caixa (Método Direto)","DFC")) %>%
    filter(DS_CONTA %in% c("Caixa Líquido Atividades Operacionais","Caixa Líquido Atividades de Investimento","Caixa Líquido Atividades de Financiamento",
                           "Aumento (Redução) de Caixa e Equivalentes")) %>%
    select(DT_REFER, DENOM_CIA, CD_CVM, GRUPO_DFP, DS_CONTA, VL_CONTA)

  #Unindo as Demonstrações em um mesmo arquivo
  DFP <- bind_rows(BPA,BPP,DRE,DFC)
  DFP <- DFP %>% select(DT_REFER, CD_CVM, DS_CONTA, VL_CONTA)

  #Transformando linhas em colunas para melhor organização e leitura dos dados
  DFP_wider <- DFP %>% pivot_wider(id_cols = c(DT_REFER, CD_CVM),names_from = DS_CONTA,values_from = VL_CONTA, values_fn = sum) %>%
    mutate(cia_ano = paste(CD_CVM,year(DT_REFER),sep = "_"),
           Liquidez_Geral = (`Ativo Circulante`+`Ativo Não Circulante`)/(`Passivo Circulante` + `Passivo Não Circulante`),
           Liquidez_Corrente = `Ativo Circulante`/`Passivo Circulante`,
           Prop_CapitalTerceiros = (`Passivo Circulante` + `Passivo Não Circulante`)/(`Passivo Circulante` + `Passivo Não Circulante` + `Patrimônio Líquido`),
           Endividamento_Geral =(`Passivo Circulante` + `Passivo Não Circulante`)/(`Ativo Circulante`+`Ativo Não Circulante`),
           Margem_Operacional = `Resultado Operacional`/`Receitas`,
           Margem_Liquida =`Lucro Líquido`/`Receitas`,
           ROE = `Lucro Líquido`/`Patrimônio Líquido`,
           FluxoCaixa_Livre = `Caixa Líquido Atividades Operacionais` + `Caixa Líquido Atividades de Investimento`) %>%
    select(17,1:16,14:25) %>%
    filter_if(~is.numeric(.), all_vars(!is.infinite(.)))

  #Convertendo os arquivos para dataframe
  DFP_long <- as.data.frame(DFP)
  DFP_wide <- as.data.frame(DFP_wider)

  #Alterando os nomes das colunas
  names(DFP_wide) <- c("cia_ano","ano","cd_cvm","ativo_total","ativo_circulante",
                            "ativo_nao_circulante","passivo_total","passivo_circulante",
                            "passivo_nao_circulante","patrimonio_liquido","receitas",
                            "resultado_operacional","lucro_liquido","fc_operacional","fc_investimento",
                            "fc_financiamento","fc_diferenca","liquidez_geral",
                            "liquidez_corrente","prop_capitalterceiros","endividamento_geral",
                            "margem_operacional","margem_liquida","roe","fluxocaixa_livre")

  DFP_wide <- DFP_wide %>% mutate(across(ativo_total:fc_diferenca,as.integer)) %>%
    mutate(across(liquidez_geral:roe, as.double)) %>%
    mutate(fluxocaixa_livre = as.integer(fluxocaixa_livre)) %>%
    distinct(cia_ano, .keep_all = TRUE)

  #Salvando os arquivo gerados e deletando os outros
  write.csv(DFP_long,"dfp_long.csv")
  write.csv(DFP_wide,"dfp_wide.csv")
  f <- list.files(path = ".", pattern = "^dfp_cia_aberta", full.names = TRUE, recursive = TRUE)
  file.remove(f)

  return(DFP_wide)
}

In [ ]:
anos <- 2017

for (ano in anos){
obter_dfp(ano)}

In [ ]:
demonstracoes <- cvm_dfp()

ℹ Using "','" as decimal and "'.'" as grouping mark. Use `read_delim()` for more control.

Rows: 44282 Columns: 15
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr  (10): CNPJ_CIA, DENOM_CIA, CD_CVM, GRUPO_DFP, MOEDA, ESCALA_MOEDA, ORDE...
dbl   (1): VERSAO
num   (1): CD_CONTA
date  (2): DT_REFER, DT_FIM_EXERC

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
ℹ Using "','" as decimal and "'.'" as grouping mark. Use `read_delim()` for more control.

Rows: 74410 Columns: 15
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr  (10): CNPJ_CIA, DENOM_CIA, CD_CVM, GRUPO_DFP, MOEDA, ESCALA_MOEDA, ORDE...
dbl   (1): VERSAO
num   (1): CD_CONTA
date  (2): DT_REFER, DT_FIM_EXERC

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_ty

In [ ]:
head(demonstracoes)

,cia_ano,DT_REFER,CD_CVM,Ativo Total,Ativo Circulante,Ativo Não Circulante,Passivo Total,Passivo Circulante,Passivo Não Circulante,Patrimônio Líquido,⋯,Caixa Líquido Atividades de Financiamento,Aumento (Redução) de Caixa e Equivalentes,Liquidez_Geral,Liquidez_Corrente,Prop_CapitalTerceiros,Endividamento_Geral,Margem_Operacional,Margem_Liquida,ROE,FluxoCaixa_Livre
,<chr>,<date>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,001023_2017,2017-12-31,001023,1353075042,47239964,1305835078,1353075042,1165740599,86096015,101238428,⋯,NA,NA,1.08087192,0.04052356,0.9251790,0.9251790,0.1139815,0.08782903,0.12125142,NA
2,014206_2017,2017-12-31,014206,14183834,1342280,12841554,14183834,10973197,1742453,1468184,⋯,NA,NA,1.11546276,0.12232351,0.8964889,0.8964889,0.1248048,0.07970463,0.14921154,NA
3,002437_2017,2017-12-31,002437,172975359,39480855,203082473,172975359,34186952,120588225,42752532,⋯,NA,NA,1.56719787,1.15485156,0.7835618,0.6380815,0.1265312,-0.04556157,-0.04036465,NA
4,014451_2017,2017-12-31,014451,3637085,1787913,3656719,3637085,1625803,1256053,834408,⋯,68819,5960,1.88927969,1.09971073,0.7754713,0.5293023,0.2003091,0.06514755,0.21240568,-62859
5,015083_2017,2017-12-31,015083,104865,36358,80947,104865,6987301,2169471,NA,⋯,NA,NA,0.01281074,0.00520344,NA,78.0595201,-3.0583981,-1.59132509,NA,NA
6,024406_2017,2017-12-31,024406,3584775,472262,3112513,3584775,3061405,132761,390609,⋯,NA,NA,1.12228826,0.15426316,0.8910367,0.8910367,0.1061145,0.09589383,0.12206836,NA


In [ ]:
#Conversão para excel
install.packages("writexl")
library(writexl)
write_xlsx(demonstracoes,"/content/dfp_wide.xlsx")

In [ ]:
# Estabelecer conexão com o banco de dados PostgreSQL
con <- dbConnect(PostgreSQL(), dbname = "seu_banco_de_dados",
                 host = "seu_host", port = "sua_porta",
                 user = "seu_usuario", password = "sua_senha")


# Inserir todas as linhas do dataframe no PostgreSQL
dbWriteTable(conn = con, name = "tabela_bd", value = demonstracoes,
             row.names = FALSE, overwrite = TRUE, append = FALSE)

# Fechar a conexão com o banco de dados
dbDisconnect(con)